Generate train, validation, and test sets from the molecule images. Construct X and Y matrices.

In [1]:
## setup cell
# import required libraries
import numpy as np
import pandas as pd
import csv
import math
import matplotlib.image as mpimg
from sklearn import model_selection
import keras

# load the list of compounds and image conversion errors
compounds = pd.read_csv("compounds.csv")
with open("errors.csv", "r") as file:
    errors = list(csv.reader(file))
errors = list(map(int, errors[0]))

# filter out the compounds that threw errors
compounds = compounds.query("SubstanceID not in @ errors")

Using TensorFlow backend.
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python

In [5]:
## test, train, validation, split
# split 80/20 into train and test sets
train, test = model_selection.train_test_split(compounds, test_size = 0.2, random_state = 1935235)

# further sudivide training set into train and validation sets to give an 80/20/20 overall split
train, validation = model_selection.train_test_split(train, test_size = 0.25, random_state = 358235)

In [ ]:
## data generator to supply images to the neural network
# declare a function molecule_image_generator
# SubstanceIDs: a list of SubstanceIDs from which to draw batches
# batch_size: the batch size
# augment: a boolean flag indicating whether data augmentation should be applied
# aug: a funtion responsible for data augmentation

def molecule_image_generator(SubstanceIDs, batch_size, augment = True, aug = None):
    # calculate the maximum number of batches from one iteration of SubstanceIDs
    max_batches = math.floor(len(SubstanceIDs) / batch_size)

    # declare a counter to track current batch
    batch = 0
    
    # loop through batches indefinitely
    while True:                
        # declare an empty array to hold the data with shape batch_size, xpixels, ypixels, channels
        X = np.empty([batch_size, 300, 300, 1], dtype = "int8")
        
        # determine which images to place in this batch
        # take batch_size observations if possible
        if batch < max_batches:
            batch_start = batch * batch_size
            batch_end = batch * batch_size + batch_size
            batch += 1
        # otherwise take as many observations as possible 
        else:
            batch_start = batch * batch_size
            batch_end = batch * batch_size + (len(SubstanceIDs) % batch_size)
            batch = 0
        
        # generate a batch of images (X)
        for i in range(batch_start, batch_end):
            # import each image into X_train; augment with a single channel dimension
            X[i, :, :, 0] = mpimg.imread("./molecule_drawings/" + SubstanceIDs[i] + ".png")
        
        # generate the corresponding batch of encoded SMILES strings (Y)
        for i in range(batch_start, batch_end):
            
        
        # yield the batch of images
        yield(X, Y)